In [14]:
import gym
import math
from gym import spaces
import numpy as np
import geopandas as gpd
from shapely.geometry import MultiPoint,Point
from matplotlib.ticker import ScalarFormatter
import matplotlib.pyplot as plt

class sea(gym.Env): 
    """
    Description:
        
        
    Observation:
        Type: Box(2),Discrete(1)
        Num     Observation               Min                     Max
        0       Vessel's Longitude        -18e6                   18e6
        1       Vessel's Latitude         -9e6                    9e6
        
        In meters (Robinson projection)
    Actions:
        Type: Discrete(8)
        Num   Action
        0     Move to the North
        1     Move to North East
        2     Move to the East
        3     Move to South East
        4     Move to the South
        5     Move to South West
        6     Move to the West
        7     Move to North West
               
    Reward:
        Reward is -1/srt(2e5^2 + 2e5^2) for every step taken in cardinal direction.
        Reward is -1 for every diagonal move.
        Reward (-c,1): (total_distance - current_position)/total_distance
        Reward is [0,-0.5] for every change of direction. 
        Reward is 100 for the termination step.
        
    Starting State:
        Assigned from the user a starting and ending point. 
    Episode Termination:
        Vessel reaches the ending point.
   
    """
     
    metadata = {'render.modes': ['human']}    
    def __init__(self,start_longitude,start_latitude,end_longitude,end_latitude):
        self.start_longitude = start_longitude
        self.start_latitude = start_latitude
        self.end_latitude = end_latitude
        self.end_longitude = end_longitude
#       distance between starting and ending port
        self.total_distance = math.sqrt((end_longitude-start_longitude)**2 + (start_latitude-start_latitude)**2)
        self.state = None
        self.previous_action = None
        self.action_space = spaces.Discrete(8)
        self.map_ = create_map()
        obs_max = [18e6,9e6]
        obs_min = [-18e6,-9e6]
        self.observation_space = spaces.Box(np.array(obs_min),np.array(obs_max),dtype=np.float64)
        
#   Compute the distance between starting and ending point
    def compute_distance(self,position):
        return math.sqrt((self.end_longitude-position[0])**2 + (self.end_latitude-position[1])**2)
    
    def reset(self):
        self.state = self.start_longitude,self.start_latitude
        self.previous_action = None
        return  np.array(self.state) 
    
    #   check whether a ship position is in the sea or land 
    def isvalid(self, position):
        return len(self.map_[self.map_.geometry.geom_almost_equals(Point(position[0],position[1]))]) != 0
    
    def step(self, action):
        if self.state == None:
            raise ValueError('Cannot call env.step() without calling reset()')
        position_x,position_y = self.state        
        if action == 0:  # move to the North
            move = self.state[0],self.state[1]+2e5
        elif action == 1:  # move to the North East
            move = self.state[0]+2e5,self.state[1]+2e5
        elif action == 2: # move to the East
            move = self.state[0]+2e5,self.state[1]
        elif action == 3: # move to the South East
            move = self.state[0]+2e5,self.state[1]-2e5
        elif action == 4: # move to the South
            move = self.state[0],self.state[1]-2e5
        elif action == 5: # move to the South West
            move = self.state[0]-2e5,self.state[1]-2e5
        elif action == 6:  # move to the West
            move = self.state[0]-2e5,self.state[1]
        elif action == 7: # move to the North West
            move = self.state[0]-2e5,self.state[1]+2e5
        else:
            raise ValueError('Actions are between 0 and 3')
        if self.isvalid(move):
            self.state = move
        done = bool(self.state == (self.end_longitude,self.end_latitude))
        a = 0.6 # reward weight
        if not done:
            if action%2 == 0: #Cardinal direction
                reward = - (1-a)*(2e5/math.sqrt(2e5**2+2e5**2)) + a*((self.total_distance - self.compute_distance(move))/self.total_distance)
            else:
                reward = -(1-a)*1 + a*((self.total_distance - self.compute_distance(move))/self.total_distance)
            if self.did_turn(action):
                reward += -self.angle_turn(action)
        else:
            reward = 100
            
        self.previous_action = action 
        return np.array(self.state), reward, done, {}
#   Check whether the vessel did turn 
    def did_turn(self,action):
        if self.previous_action == None:
            return False
        elif self.previous_action == action :
            return False
        else:
            return True        
#   Mesure the turning angle
    def angle_turn(self,action):
        return abs((action/8-self.previous_action/8))
    
    
    def render(self,mode='human'):
        fig, ax = plt.subplots(figsize=(20, 20))
        self.map_.plot(ax=ax)
        point = Point(self.state[0],self.state[1])
        start = Point(self.start_longitude,self.start_latitude)
        end = Point(self.end_longitude,self.end_latitude)

        ax.scatter(point.x,point.y,color='red',marker='*',s=150,label='Ship\'s location')
        ax.scatter(start.x,start.y,color='black',s=150,label='Starting Port')
        ax.scatter(end.x,end.y,color='orange',s=150,label='Ending Port')
        

        ax.set_title("Ocean grid: Robinson Coordinate Reference System,",fontsize = 20)
        ax.set_xlabel("X Coordinates (meters)",fontsize = 20)
        ax.set_ylabel("Y Coordinates (meters)",fontsize = 20)
        ax.legend()

        for axis in [ax.xaxis, ax.yaxis]:
            formatter = ScalarFormatter()
            formatter.set_scientific(False)
            axis.set_major_formatter(formatter)
        plt.show()

# Create the grid
def create_map(n=5e-6):
    ocean = gpd.read_file('C://Users//potis//Desktop//b//ne_10m_ocean_scale_rank.shp')
    ocean = ocean.to_crs('ESRI:54030')
    result=[]
    for i in range(len(ocean)):
        xmin, ymin, xmax, ymax = ocean.bounds.loc[i].T.values
        x = np.arange(np.floor(xmin * n) / n, np.ceil(xmax * n) / n, 1 / n)
        y = np.arange(np.floor(ymin * n) / n, np.ceil(ymax * n) / n, 1 / n)    

        grid = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
        points = MultiPoint(grid)

        if ocean.geometry[i].is_valid:
            result.append(points.intersection(ocean.geometry.loc[i]))
        else:
            result.append(points.intersection(ocean.geometry.loc[i].buffer(0)))
        results = [j for i in result for j in i]
    return gpd.GeoDataFrame(results,columns=['geometry'],crs=ocean.crs)

In [9]:
import numpy as np
import gym
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Input,Dropout, BatchNormalization,Reshape
from keras.optimizers import Adam
from tensorflow.keras.models import Model 

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

# Initialise the environment
env = sea(-600000.000-2e5, 6000000.000,-5000000.000, 6e5)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Deep q learning model
input = Input(shape=(1,)+ env.observation_space.shape, name='Input')
x = input
x = Reshape((2,))(x)
x = BatchNormalization()(x)
x = Dense(units= 2*8,kernel_initializer='glorot_uniform', activation='relu')(x)
x = Dense(units= 2*8,kernel_initializer='glorot_uniform', activation='relu')(x)
x = Dense(units= 2*8,kernel_initializer='glorot_uniform', activation='relu')(x)
output = Dense(units=nb_actions, kernel_initializer='glorot_uniform', activation='linear', name='Output')(x)
model = Model(inputs=input, outputs=output)
print(model.summary())

# Expirience replay
memory = SequentialMemory(limit=50000, window_length=1)
 
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,enable_double_dqn=False, nb_steps_warmup=150,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3),  metrics=['mse'])


history = dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

# dqn.test(env, nb_episodes=1, visualize=False, verbose=1)

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 1, 2)]            0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 2)                 0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 2)                 8         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                48        
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 16)                272       
_________________________________________________________________
Output (Dense)               (None, 8)               

In [15]:
history.history['episode_reward']

[-4516.565059144794,
 -94.35162657943295,
 -121.90276346497836,
 -169.2546826382864,
 -280.0664897405679,
 -159.80278066380617,
 -278.1070872754664,
 -147.4784823604671,
 -78.92420477102584,
 -227.16335438134075,
 -412.26932713623796,
 -485.78413386236036,
 -97.25999214550919,
 -41.965644896148746,
 -69.87369482739086,
 -26.25117146060849,
 2.1384085561852686,
 11.679985284018528,
 51.58169398043035,
 37.32471324501808,
 33.967491131329595,
 26.706318042244504,
 21.96523792047779,
 15.109956101197412,
 31.05201063298749,
 -1.2357538258010266,
 53.5692579050378,
 19.118165847850577,
 -19.853472825300386,
 58.424910295559,
 54.01190683768653,
 35.634174789199335,
 65.96790466706753,
 51.0328030409626,
 5.3647037653390015,
 31.344517125153928,
 -350.1066886436787,
 37.15877245403425,
 -318.5542024735754,
 51.75727190873015,
 -90.07049476936868,
 26.13346274376684,
 38.38854328547263,
 63.33939483347396,
 -40.45528319788835,
 -685.870945167121,
 -375.8914489883975,
 74.74259962779243,
 53.